In [1]:
!pip install fitz
!pip install annoy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552446 sha256=fccacab396a3ead3f4a41c1f68be8c5655192f70406c002eb04a299b115919d1
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [2]:
!pip install --upgrade fitz
!pip install --upgrade PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 57.8 MB/s eta 0:00:00


In [3]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
import fitz
from nltk.tokenize import sent_tokenize, word_tokenize
from sentence_transformers import SentenceTransformer, util
import annoy

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# Load pre-trained sentence embedding model (e.g., Universal Sentence Encoder)
model = SentenceTransformer("all-mpnet-base-v2")

def parse_pdf(pdf_path):
    # Open PDF with fitz (or other parsing library if needed)
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()  # Handle layout variations appropriately
    return text

def preprocess_text(text):
    sentences = sent_tokenize(text)
    sentences = [word_tokenize(s) for s in sentences]  # Handle language-specific issues
    return sentences

def create_vector_db(sentences, model):
    sentence_embeddings = model.encode(sentences)

    # Create VectorDB using annoy
    num_embeddings = len(sentence_embeddings)
    dim = sentence_embeddings[0].shape[0]  # Get embedding dimension
    vector_db = annoy.AnnoyIndex(f=dim)

    # Add embeddings to the index
    for i, embedding in enumerate(sentence_embeddings):
        vector_db.add_item(i, embedding)

    # Build the index (requires some time for larger datasets)
    vector_db.build(10)  # Number of trees in the index (experiment with this)

    return vector_db

def semantic_search(query, vector_db, model):
    query_embedding = model.encode(query)

    # Perform nearest neighbor search with annoy
    nearest_neighbors, *distances = vector_db.get_nns_by_vector(query_embedding, n=5, include_distances=True)

    return nearest_neighbors, distances

def retrieve_answer(pdf_path, query, vector_db, model):
    text = parse_pdf(pdf_path)
    sentences = preprocess_text(text)
    embeddings = model.encode(sentences)

    # Create VectorDB if not already created
    if not vector_db:
        vector_db = create_vector_db(sentences, model)

    nearest_neighbors, distances = semantic_search(query, vector_db, model)
    answer_sentences = []
    for i, idx in enumerate(nearest_neighbors):
        # Join the elements of the list into a single string
        sentence = ' '.join(sentences[idx])
        # Append the string to the list
        answer_sentences.append(sentence + ".")

    return answer_sentences, distances

# Example usage
pdf_path = "musiccc.pdf"
query = "Within small musical groups what is crucial for their functioning"
vector_db = None  # Initialize VectorDB only once

answer, distances = retrieve_answer(pdf_path, query, vector_db, model)

for i in range(len(answer)):
  print(f"{i+1}.{answer[i]}")
#print(f"Distances to the nearest sentences: {distances}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

1.Group music making is also beneficial to the development of social skills and can contribute to health and well-being throughout the lifespan and can therefore contribute to community cohesion providing benefits to society as a whole ..
2.Musicians playing from notation are constantly required to adopt quasi-mathematical processes to sub-divide beats and turn rhythmic notation into sound ..
3.Music participation enhances measured creativity , particularly when the musical activity itself is creative , for instance , improvisation ..
4.Musicians with similar observable skills may have developed different approaches to developing them which may or may not facilitate transfer to other tasks ..
5.Music involvement was found to vary systematically by class and gender status , and such involvement had implications for both mathematics and reading achievement and for young children and adolescents ..


<ipython-input-6-fa43166ba7c6>:23: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  vector_db = annoy.AnnoyIndex(f=dim)
